### Clean up first

In [1]:
import sys.process._
"rm -rf /tmp/hudi/".!!
"mkdir -p /tmp/hudi".!!
lazy val result3 = "tree -a /tmp/hudi/".!!
println(result3)


/tmp/hudi/

0 directories, 0 files



import sys.process._
res1_1: String = ""
res1_2: String = ""
result3: String = [lazy]

### Setup project

In [2]:
import $ivy.`org.apache.hudi:hudi-spark3.3-bundle_2.12:0.14.0`
// import $ivy.`org.apache.hudi:hudi-common:1.0.0`
import $ivy.`org.apache.spark:spark-sql_2.12:3.3.2`
import $ivy.`org.apache.spark:spark-avro_2.12:3.3.2`
import $ivy.`org.apache.spark:spark-hive_2.12:3.3.2`
import $cp.`CustomMergeIntoConnector.jar`

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.hudi.QuickstartUtils._
import org.apache.spark.sql.hive.HiveExternalCatalog
import scala.collection.JavaConversions._
import org.apache.spark.sql.SaveMode._
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.common.table.HoodieTableConfig._
import org.apache.hudi.config.HoodieWriteConfig._
import org.apache.hudi.keygen.constant.KeyGeneratorOptions._
import org.apache.hudi.common.model.HoodieRecord
import com.payloads.CustomMergeIntoConnector

val spark = SparkSession.builder()
  .appName("HudiLocalSession")
  .master("local[1]")  // Runs on local machine with 1 local worker / thread, so we can see the impact on COPY_ON_WRITE
  .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
  .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog")
  .config("spark.kryo.registrator", "org.apache.spark.HoodieSparkKryoRegistrar")
  .config("spark.driver.extraJavaOptions", "-Dscala.repl.maxprintstring=0")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark with Hudi is ready!")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


25/03/04 00:17:00 WARN Utils: Your hostname, DESKTOP-M94RUSC resolves to a loopback address: 127.0.1.1; using 172.17.75.227 instead (on interface eth0)
25/03/04 00:17:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/04 00:17:01 INFO SparkContext: Running Spark version 3.3.2
25/03/04 00:17:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/04 00:17:01 INFO ResourceUtils: ==============================================================
25/03/04 00:17:01 INFO ResourceUtils: No custom resources configured for spark.driver.
25/03/04 00:17:01 INFO ResourceUtils: ==============================================================
25/03/04 00:17:01 INFO SparkContext: Submitted application: HudiLocalSession
25/03/04 00:17:01 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024

import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $cp.$
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.hudi.QuickstartUtils._
import org.apache.spark.sql.hive.HiveExternalCatalog
import scala.collection.JavaConversions._
import org.apache.spark.sql.SaveMode._
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.common.table.HoodieTableConfig._
import org.apache.hudi.config.HoodieWriteConfig._
import org.apache.hudi.keygen.constant.KeyGeneratorOptions._
import org.apache.hudi.common.model.HoodieRecord
import com.payloads.CustomMergeIntoConnector
spark: SparkSession = org.apache.spark.sql.SparkSession@6dd4ec64

### Create and Insert Table

In [3]:
val columns = Seq("ts","uuid","rider","driver","fare","city")
val data =
  Seq((1695159649087L,"334e26e9-8355-45cc-97c6-c31daf0df330","rider-A","driver-K",19.10,"san_francisco"),
    (1695091554788L,"e96c4396-3fad-413a-a942-4cb36106d721","rider-C","driver-M",27.70 ,"san_francisco"),
    (1695046462179L,"9909a8b1-2d15-4d3d-8ec9-efc48c536a00","rider-D","driver-L",33.90 ,"san_francisco"),
    (1695516137016L,"e3cf430c-889d-4015-bc98-59bdce1e530c","rider-F","driver-P",34.15,"sao_paulo"    ),
    (1695115999911L,"c8abbe79-8d89-47ea-b4ce-4d224bae5bfa","rider-J","driver-T",17.85,"chennai"));

val tableName = "trips_table"
val basePath = "file:///tmp/hudi/trips_table"
var inserts = spark.createDataFrame(data).toDF(columns:_*)

inserts.write.format("hudi").
  option("hoodie.datasource.write.partitionpath.field", "city").
  option("hoodie.datasource.write.hive_style_partitioning", "true"). // Force hive style dir naming for partitions.
  option("hoodie.datasource.write.storage.type", "COPY_ON_WRITE").
  option("hoodie.table.name", tableName).
  mode(Overwrite).
  save(basePath)



# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf
# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]


columns: Seq[String] = List("ts", "uuid", "rider", "driver", "fare", "city")
data: Seq[(Long, String, String, String, Double, String)] = List(
  (
    1695159649087L,
    "334e26e9-8355-45cc-97c6-c31daf0df330",
    "rider-A",
    "driver-K",
    19.1,
    "san_francisco"
  ),
  (
    1695091554788L,
    "e96c4396-3fad-413a-a942-4cb36106d721",
    "rider-C",
    "driver-M",
    27.7,
    "san_francisco"
  ),
  (
    1695046462179L,
    "9909a8b1-2d15-4d3d-8ec9-efc48c536a00",
    "rider-D",
    "driver-L",
    33.9,
    "san_francisco"
  ),
  (
    1695516137016L,
    "e3cf430c-889d-4015-bc98-59bdce1e530c",
    "rider-F",
    "driver-P",
    34.15,
    "sao_paulo"
  ),
  (
    1695115999911L,
    "c8abbe79-8d89-47ea-b4ce-4d224bae5bfa",
    "rider-J",
    "driver-T",
    17.85,
...
tableName: String = "trips_table"
basePath: String = "file:///tmp/hudi/trips_table"
inserts: DataFrame = [ts: bigint, uuid: string ... 4 more fields]

* If we check the Hudi table dir, we can find several parquet files.
* Here we have 1 parquet file for each city/partition.
* This is because we set the number of worker to 1 and the file is tiny.

In [4]:
import spark.implicits._
val updatesDf = spark.read.format("hudi").load(basePath).filter($"rider" === "rider-D").withColumn("fare", col("fare") * 10)

updatesDf.write.format("hudi").
  option("hoodie.datasource.write.operation", "upsert").
  option("hoodie.datasource.write.partitionpath.field", "city").
  option("hoodie.table.name", tableName).
  mode(Append).
  save(basePath)

import spark.implicits._
updatesDf: DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 9 more fields]

In [5]:
val adjustedFareDF = spark.read.format("hudi").
  load(basePath).limit(2).
  withColumn("fare", col("fare") * 10)

adjustedFareDF.write.format("hudi").
  option("hoodie.datasource.write.payload.class","com.payloads.CustomMergeIntoConnector").
  mode(Append).
  save(basePath)
// Notice Fare column has been updated but all other columns remain intact.
spark.read.format("hudi").load(basePath).show()

+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|           ts|                uuid|  rider|  driver| fare|         city|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+
|  20250304001809024|20250304001809024...|20250304001707251...|    city=san_francisco|14e71d60-c51e-443...|1695159649087|334e26e9-8355-45c...|rider-A|driver-K|210.1|san_francisco|
|  20250304001809024|20250304001809024...|20250304001707251...|    city=san_francisco|14e71d60-c51e-443...|1695091554788|e96c4396-3fad-413...|rider-C|driver-M|304.7|san_francisco|
|  20250304001754658|20250304001754658...|20250304001707251...|    city=san_francisco|14e71d60-c51e-

adjustedFareDF: DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 9 more fields]

In [6]:
// Lets  delete rider: rider-D
val deletesDF = spark.read.format("hudi").load(basePath).filter($"rider" === "rider-F")

deletesDF.write.format("hudi").
  option("hoodie.datasource.write.operation", "delete").
  option("hoodie.datasource.write.partitionpath.field", "city").
  option("hoodie.table.name", tableName).
  mode(Append).
  save(basePath)

deletesDF: Dataset[Row] = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 9 more fields]

In [7]:
spark.stop()